In [16]:
import numpy as np
import pandas as pd
from sklearn import neural_network, linear_model, cluster
np.random.seed(0)

a = pd.read_csv('healthcare.csv', header=0,names=['gender', 'age', 'hypertension', 'heart_disease',
                       'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level',
                       'bmi', 'smoking_status', 'stroke'])

#dropping the columns
a = a.drop('bmi', axis=1)
a = a.drop('avg_glucose_level', axis=1)

#convert numpy

In [17]:
#ever_married should be indicator type; so yes=1 and no=0 
a.loc[a['ever_married']=='Yes', 'ever_married'] = '1'
a.loc[a['ever_married']=='No', 'ever_married'] = '0'
print(a)


       gender   age  hypertension  heart_disease ever_married      work_type  \
9046     Male  67.0             0              1            1        Private   
51676  Female  61.0             0              0            1  Self-employed   
31112    Male  80.0             0              1            1        Private   
60182  Female  49.0             0              0            1        Private   
1665   Female  79.0             1              0            1  Self-employed   
...       ...   ...           ...            ...          ...            ...   
18234  Female  80.0             1              0            1        Private   
44873  Female  81.0             0              0            1  Self-employed   
19723  Female  35.0             0              0            1  Self-employed   
37544    Male  51.0             0              0            1        Private   
44679  Female  44.0             0              0            1       Govt_job   

      Residence_type   smoking_status  

In [18]:
full=a
for col in ['gender', 'work_type', 'Residence_type', 'smoking_status']:
  prefix_col = col
  full = pd.concat([full, pd.get_dummies(full[col], prefix=prefix_col, drop_first=False)], axis=1)
  full = full.drop(col, axis=1)
print(full)
print(list(full.columns))

        age  hypertension  heart_disease ever_married  stroke  gender_Female  \
9046   67.0             0              1            1       1              0   
51676  61.0             0              0            1       1              1   
31112  80.0             0              1            1       1              0   
60182  49.0             0              0            1       1              1   
1665   79.0             1              0            1       1              1   
...     ...           ...            ...          ...     ...            ...   
18234  80.0             1              0            1       0              1   
44873  81.0             0              0            1       0              1   
19723  35.0             0              0            1       0              1   
37544  51.0             0              0            1       0              0   
44679  44.0             0              0            1       0              1   

       gender_Male  gender_Other  work_

In [19]:
full_np = full.to_numpy()
#x and y values
y = (full_np[:, -1]).astype(np.float32)
x = np.delete(full_np, [full_np.shape[1]-1], axis=1).astype(np.float32)
trn_x, trn_y = x[:3000], y[:3000]
tst_x, tst_y = x[3000:], y[3000:]


In [20]:
#separate 0s and 1s 
trn_zero_inds = np.where(trn_y==0)[0]
trn_one_inds = np.where(trn_y==1)[0]
tst_zero_inds = np.where(tst_y==0)[0]
tst_one_inds = np.where(tst_y==1)[0]

#collecting data with zeros
trn_zeros = np.random.choice(trn_zero_inds.shape[0], trn_one_inds.shape[0], replace=True)
tst_zeros = np.random.choice(tst_zero_inds.shape[0], tst_one_inds.shape[0], replace=True)

#dividing equal number of 0s and 1s 
trn_x = np.concatenate((trn_x[trn_zeros], trn_x[trn_one_inds]), axis=0)
tst_x = np.concatenate((tst_x[tst_zeros], tst_x[tst_one_inds]), axis=0)
trn_y = np.concatenate((trn_y[trn_zeros], trn_y[trn_one_inds]), axis=0)
tst_y = np.concatenate((tst_y[tst_zeros], tst_y[tst_one_inds]), axis=0)


trn_shuffle = np.random.choice(trn_x.shape[0], trn_x.shape[0], replace=False)
trn_x, trn_y = trn_x[trn_shuffle], trn_y[trn_shuffle]
trn_size = trn_x.shape[0]//2

trn_x, trn_y, ho_x, ho_y = trn_x[:trn_size], trn_y[:trn_size], trn_x[trn_size:], trn_y[trn_size:]

pois_rates = [0.5, 1, 2]


In [21]:
lm = linear_model.LogisticRegression()
lm.fit(trn_x, trn_y)
lm_preds = np.multiply(lm.predict_proba(ho_x), np.eye(2)[ho_y.astype(np.int)]).sum(axis=1)
lm.score(tst_x, tst_y)

nn = neural_network.MLPClassifier(hidden_layer_sizes=(10,))
nn.fit(trn_x, trn_y)
nn_preds = np.multiply(nn.predict_proba(ho_x), np.eye(2)[ho_y.astype(np.int)]).sum(axis=1)
clean_tst_acc = nn.score(tst_x, tst_y)
clean_tst_acc

np.multiply(nn.predict_proba(tst_x), np.eye(2)[tst_y.astype(int)]).mean()*2

np.random.seed(0)

#2:13 colums for subclasses
protected = np.concatenate((ho_x[:, 2:13], ho_x[:, 2:13]), axis=1)
tst_prot = np.concatenate((tst_x[:, 2:13], tst_x[:, 2:13]), axis=1)
trn_prot = np.concatenate((trn_x[:, 2:13], trn_x[:, 2:13]), axis=1)

all_cols = list(full.columns)
prot_cols = all_cols[2:13]
subclasses, counts = np.unique(protected, axis=0, return_counts=True)

hd_sbcl_conf = []
hd_used = []
hd_lr_errs = np.zeros((len(subclasses), 3, len(pois_rates)))  # subcl, (clean_acc, collat, target), pois_ind
hd_nn_errs = np.zeros((len(subclasses), 3, len(pois_rates)))  # subcl, (clean_acc, collat, target), pois_ind


c:\users\abhin\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\abhin\appdata\local\programs\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:
for i, (subcl, count) in enumerate(zip(subclasses, counts)):
  if count > 10 and count < 100:
    hd_used.append((i, count))
    tst_sbcl = np.where(np.linalg.norm(tst_prot - subcl, axis=1)==0)
    sbcl = np.where(np.linalg.norm(protected-subcl, axis=1)==0)
    trn_sbcl = np.where(np.linalg.norm(trn_prot - subcl, axis=1)==0)
    p_t_x, p_t_y = tst_x[tst_sbcl], tst_y[tst_sbcl]
    pois_x_base, pois_y_base = ho_x[sbcl], ho_y[sbcl]
    sc_lr_pred, sc_nn_pred = lm_preds[sbcl].mean(), nn_preds[sbcl].mean()
    trn_ct = trn_sbcl[0].shape[0]
    tst_ct = p_t_x.shape[0]
    print(sc_lr_pred, sc_nn_pred)
    if len(p_t_x) !=0:
        hd_sbcl_conf.append(2*np.multiply(lm.predict_proba(p_t_x),
                                      np.eye(2)[p_t_y.astype(int)]).mean())
    
    all_errs = []
    if len(p_t_x) !=0:
        for j, pois_ct in enumerate([int(trn_ct*pois_rate) for pois_rate in pois_rates]):
          pois_inds = np.random.choice(pois_x_base.shape[0], pois_ct, replace=True)
          pois_x, pois_y = pois_x_base[pois_inds], 1-pois_y_base[pois_inds]

          full_x, full_y = np.concatenate((trn_x, pois_x), axis=0), np.concatenate((trn_y, pois_y), axis=0)
          print([prot_col for v, prot_col in zip(subcl, prot_cols) if v > 0.5], trn_ct, pois_ct, tst_ct)
          print("poison fraction:", pois_ct/trn_ct)
            
          lmps = [linear_model.LogisticRegression(solver='liblinear', max_iter=500) for _ in range(3)]
          for lmp in lmps:
            lmp.fit(full_x, full_y)
          nnps = [neural_network.MLPClassifier(hidden_layer_sizes=(10,), max_iter=3000) for _ in range(3)]
          for nnp in nnps:
            nnp.fit(full_x, full_y)
          
          lmp_acc = np.mean([lmp.score(tst_x, tst_y) for lmp in lmps])
          print("LR Acc on poisoned dataset {:.3f}".format(lmp_acc))
    
          lmc_sbc = lm.score(p_t_x, p_t_y)
          print("LR Clean Subclass accuracy {:.3f}".format(lmc_sbc))

          lmp_sbc = np.mean([lmp.score(p_t_x, p_t_y) for lmp in lmps])
          print("LR Posioned subclass accuray {:.3f}".format(lmp_sbc))
            
          nnp_acc = np.mean([nnp.score(tst_x, tst_y) for nnp in nnps])
          print("NN Acc on poisoned dataset {:.3f}".format(nnp_acc))

          nnc_sbc = nn.score(p_t_x, p_t_y)
          print("NN Clean Subclass accuracy {:.3f}".format(nnc_sbc))

          nnp_sbc = np.mean([nnp.score(p_t_x, p_t_y) for nnp in nnps])
          print("NN Posioned subclass accuray {:.3f}".format(nnp_sbc))

          hd_nn_errs[i, 0, j] = nnc_sbc
          hd_nn_errs[i, 1, j] = nnp_acc
          hd_nn_errs[i, 2, j] = nnp_sbc
        
          hd_lr_errs[i, 0, j] = lmc_sbc
          hd_lr_errs[i, 1, j] = lmp_acc
          hd_lr_errs[i, 2, j] = lmp_sbc


0.9648209581168505 0.7186747804284096
['gender_Male', 'work_type_children'] 15 7 20
poison fraction: 0.4666666666666667
LR Acc on poisoned dataset 1.000
LR Clean Subclass accuracy 1.000
LR Posioned subclass accuray 1.000
NN Acc on poisoned dataset 1.000
NN Clean Subclass accuracy 1.000
NN Posioned subclass accuray 1.000
['gender_Male', 'work_type_children'] 15 15 20
poison fraction: 1.0
LR Acc on poisoned dataset 1.000
LR Clean Subclass accuracy 1.000
LR Posioned subclass accuray 1.000
NN Acc on poisoned dataset 0.996
NN Clean Subclass accuracy 1.000
NN Posioned subclass accuray 0.883
['gender_Male', 'work_type_children'] 15 30 20
poison fraction: 2.0
LR Acc on poisoned dataset 1.000
LR Clean Subclass accuracy 1.000
LR Posioned subclass accuray 1.000
NN Acc on poisoned dataset 0.982
NN Clean Subclass accuracy 1.000
NN Posioned subclass accuray 0.450
0.9348363009974408 0.5827837038040161
['gender_Male', 'work_type_Private'] 29 14 43
poison fraction: 0.4827586206896552
LR Acc on poisoned

In [23]:
hd_lr_errs  # subcl, (clean_acc, collat, target), pois_ind
hd_nn_errs  # subcl, (clean_acc, collat, target), pois_ind

hd_targets = np.zeros((len(hd_used), 3, 2))

for j, ((i, count), conf) in enumerate(zip(hd_used, hd_sbcl_conf)):
  for pois_ind in range(3):
    #print(i, count, conf, pois_ind)
    this_err = hd_nn_errs[i, :, pois_ind]
    clean_acc, collat, target = this_err[0], this_err[1], this_err[2]
    hd_targets[j, pois_ind, 0] = clean_acc - target
    hd_targets[j, pois_ind, 1] = collat

In [24]:
for pois_ind in range(3): # for each pois_ind/pois frac
    print("\n")
    
    # print the poison index and poison rate
    print("Pois Index: %d, Pois fraction: %f of identical training samples " %( pois_ind, pois_rates[pois_ind]))
    
    # sort the target metric scores for current poison index
    sorted_hd_targets = np.argsort(hd_targets[:, pois_ind, 0])
    
    # take the last index of sorted_hd_targets as it is the index with highest target score for current poison index
    top1_index = sorted_hd_targets[-1]
    
#     print("Hd error/used Index: ", top1_index) # print the top1_index

    # get the subclasses index, count for the feature with the highest target score using top1_index, 
    # the key is hd_target and hd_used have same indexing, so top1_index which is index in hd_targets is also the same index
    # in hd_used, then hd_used has the index and count of the subclass to which the highest target score belongs to
    subclasses_ind, count = hd_used[top1_index]
    print("Subclass Index: %d, Count: %d" % (subclasses_ind, count))
    
    subclass = subclasses[subclasses_ind] # get the subclass to which the highest score belongs to
    print([prot_col for v, prot_col in zip(subclass, prot_cols) if v > 0.5]) # print the subclass
    print(subclass)
        
    # print its target and collat values
    print("Target: %f, Collat: %f" % (hd_targets[top1_index, pois_ind, 0], hd_targets[top1_index, pois_ind, 1]))



Pois Index: 0, Pois fraction: 0.500000 of identical training samples 
Subclass Index: 4, Count: 12
['gender_Female', 'work_type_children']
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
Target: 0.015873, Collat: 0.998904


Pois Index: 1, Pois fraction: 1.000000 of identical training samples 
Subclass Index: 4, Count: 12
['gender_Female', 'work_type_children']
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
Target: 0.603175, Collat: 0.979167


Pois Index: 2, Pois fraction: 2.000000 of identical training samples 
Subclass Index: 4, Count: 12
['gender_Female', 'work_type_children']
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
Target: 0.825397, Collat: 0.971491
